In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


# Read the data

In [2]:
df = pd.read_csv('housing_preprocessed.csv')
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,-5.064036,18.0,0.837248,0,0.538,6.575,65.2,1.408545,0.000000,296.0,15.3,396.90,1.605430,24.0
1,-3.600502,0.0,1.955860,0,0.469,6.421,78.9,1.602836,0.693147,242.0,17.8,396.90,2.212660,21.6
2,-3.601235,0.0,1.955860,0,0.469,7.185,61.1,1.602836,0.693147,242.0,17.8,392.83,1.393766,34.7
3,-3.430523,0.0,0.779325,0,0.458,6.998,45.8,1.802073,1.098612,222.0,18.7,394.63,1.078410,33.4
4,-2.672924,0.0,0.779325,0,0.458,7.147,54.2,1.802073,1.098612,222.0,18.7,396.90,1.673351,36.2


In [3]:
df.shape

(506, 14)

In [4]:
dataset = df.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

In [5]:
# define base model
def baseline_model():
  # create model
  model = Sequential()
  model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [6]:
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -40.10 (34.76) MSE


In [10]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -34.12 (36.20) MSE


In [8]:
# define the model
def larger_model():
  # create model
  model = Sequential()
  model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
  model.add(Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [9]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -24.59 (26.84) MSE


In [11]:
def wider_model():
  # create model
  model = Sequential()
  model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='relu'))
  model.add(Dense(10, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  print(model.summary())
  return model

In [12]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=10, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 20)                280       
_________________________________________________________________
dense_42 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 11        
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
455/455 [==============================] - 0s 670us/step - loss: 595.3878
Epoch 2/10
455/455 [==============================] - 0s 356us/step - loss: 417.0710
Epoch 3/10
455/455 [==============================] - 0s 391us/step - loss: 138.8981
Epoch 4/10
455/455 [==============================] - 0s 449us/step - loss: 78.9136
Epoch 5/10
455/455 [=====================

456/456 [==============================] - 0s 263us/step - loss: 29.7464
Epoch 10/10
50/50 [==============================] - 0s 622us/step
Wider: -44.10 (43.22) MSE
